In [25]:
import time
import logging
from kiteconnect import KiteTicker
logger = logging.getLogger()
logging.basicConfig(level=logging.DEBUG)
import datetime
from utils.RenkoUtil import *
from classes.DayEntry import Day
from classes.MinuteEntry import Minute
from classes.TradeEntry import Trade
from classes.RenkoEntry import Renko
from collections import deque
import thread
# import csv
import pandas as pd

In [26]:
#initialize
money = 1
day = Day(0, money, False)
stepCount = 3
brickHeight = .12
renkoDeque = deque(maxlen=stepCount)
printRenkoDeque(renkoDeque)

DEBUG:root:empty queue


In [27]:
# df = pd.read_csv('tick_adani_15-04.csv')
from utils.PreProcess import basicPreProcessing
df = basicPreProcessing("IntradayData_2018", "ADANIENT")

In [28]:
df[:10]

,stock,date,time,openingPrice,high,low,closingPrice,volume
0,ADANIENT,20180101,09:16,166.00,166.20,165.75,166.00,14667
1,ADANIENT,20180101,09:17,166.15,166.60,165.95,166.50,8280
2,ADANIENT,20180101,09:18,166.60,166.60,166.30,166.40,13583
3,ADANIENT,20180101,09:19,166.45,166.50,166.30,166.35,8442
4,ADANIENT,20180101,09:20,166.15,166.50,166.15,166.40,22428
5,ADANIENT,20180101,09:21,166.20,166.55,166.20,166.35,8921
6,ADANIENT,20180101,09:22,166.30,166.50,166.15,166.25,6613
7,ADANIENT,20180101,09:23,166.25,166.40,166.20,166.40,5926
8,ADANIENT,20180101,09:24,166.20,166.40,166.20,166.20,5987
9,ADANIENT,20180101,09:25,166.25,166.25,166.15,166.15,3368


In [29]:
# newRenko = Renko(1242, 1243.2, -1, df['timestamp'][0])
# renkoDeque.append(newRenko)
# printRenkoDeque(renkoDeque)

In [30]:
def tickScript(tick):
    verbose = False
    global day
    global renkoDeque
    global stepCount
    global brickHeight
#     logging.info('Inside tickfunction that one')
    minute = Minute(tick['timestamp'], tick['last_price'])
    renkoDeque, newRenko = generateRenko(minute, renkoDeque, brickHeight, stepCount, verbose)
    while newRenko:
        printRenkoDeque(renkoDeque)
        print('\n')
        if len(renkoDeque) == stepCount:
    #             if marketOpen <= minute.time < marketClose:
            logging.info("Boughtflag: {}".format(day.boughtFlag))
            if day.boughtFlag == 0:
                #market enter logic
                if getEmotion(renkoDeque) == stepCount:
                    #buy
                    day.boughtFlag = 1
                    day.currentTrade = Trade(minute.listPrice, minute.time, 0, 0, day.boughtFlag)
                    logging.info('Long trade {}'.format(day.currentTrade.enterPrice))

                elif getEmotion(renkoDeque) == -1 * stepCount:
                    #sell
                    day.boughtFlag = -1
                    day.currentTrade  = Trade(minute.listPrice, minute.time, 0, 0, day.boughtFlag)
                    logging.info('Short trade {}'.format(day.currentTrade.enterPrice))

            elif day.boughtFlag == 1:
                #market exit logic (when already bought)
                if renkoExitLogic(renkoDeque, stepCount, day.boughtFlag):
                    logging.info('exiting trade, enter: {} exit: {}'.format(day.currentTrade.enterPrice, minute.listPrice))
                    day.currentTrade.exitTrade('renkoExit', minute.time, minute.listPrice, verbose = True)
                    day.addTrade(day.currentTrade)
                    logging.info('Long exit')
            else:
                #market exit logic (when already sold)
                if renkoExitLogic(renkoDeque, stepCount, day.boughtFlag):
                    logging.info('exiting trade, enter: {} exit: {}'.format(day.currentTrade.enterPrice, minute.listPrice))
                    day.currentTrade.exitTrade('renkoExit', minute.time, minute.listPrice, verbose = True)
                    day.addTrade(day.currentTrade)
                    logging.info('Short exit')
        renkoDeque, newRenko = generateRenko(minute, renkoDeque, brickHeight, stepCount, verbose)
    #         elif minute.time >= marketClose and day.sellEndOfDay:
    #             if day.boughtFlag == 1:
    #change to currentTrade
    #                 trade.exitTrade('eod', minute.time, minute.closingPrice, verbose)
    #                 day.addTrade(trade)
    #             elif day.boughtFlag == -1:
    #                 trade.exitTrade('eod', minute.time, minute.closingPrice, verbose)
    #                 day.addTrade(trade)

In [ ]:
for row in df.iterrows():
    tick = {}
    tick['timestamp'] = row[1].time
    tick['last_price'] = row[1].closingPrice
#     logging.debug("Ticks: {} {}".format(tick['timestamp'], tick['last_price']))
    try:
        tickScript(tick)
    except:
        logging.debug('Unable to start that one thread')
#     time.sleep(0.1)

INFO:root:Boughtflag: 0
INFO:root:Boughtflag: 0
INFO:root:Long trade 166.5
INFO:root:Boughtflag: 1
DEBUG:root:emotion : 3
INFO:root:Boughtflag: 1
DEBUG:root:emotion : 1
DEBUG:root:renkoExit : True
INFO:root:exiting trade, enter: 166.5 exit: 166.2
DEBUG:root:Inside exitTrade fn
DEBUG:root:Inside addTrade to day fn
DEBUG:root:money 0.997998558559
DEBUG:root:after reset bought flag : 0
INFO:root:Long exit
INFO:root:Boughtflag: 0
INFO:root:Boughtflag: 0
INFO:root:Short trade 165.55
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -3
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -3
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -3
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -1
DEBUG:root:renkoExit : True
INFO:root:exiting trade, enter: 165.55 exit: 165.95
DEBUG:root:Inside exitTrade fn
DEBUG:root:Inside addTrade to day fn
DEBUG:root:money 0.995393899591
DEBUG:root:after reset bought flag : 0
INFO:root:Short exit
INFO:root:Boughtflag: 0
INFO:root:Boughtflag: 0
INFO:root:Long trade 166.15
INFO:

time: 09:16, low: 166.0, high: 166.0, type: 0


time: 09:16, low: 166.0, high: 166.0, type: 0
time: 09:17, low: 166.0, high: 166.12, type: 1


time: 09:16, low: 166.0, high: 166.0, type: 0
time: 09:17, low: 166.0, high: 166.12, type: 1
time: 09:17, low: 166.12, high: 166.24, type: 1


time: 09:17, low: 166.0, high: 166.12, type: 1
time: 09:17, low: 166.12, high: 166.24, type: 1
time: 09:17, low: 166.24, high: 166.36, type: 1


time: 09:17, low: 166.12, high: 166.24, type: 1
time: 09:17, low: 166.24, high: 166.36, type: 1
time: 09:17, low: 166.36, high: 166.48, type: 1


time: 09:17, low: 166.24, high: 166.36, type: 1
time: 09:17, low: 166.36, high: 166.48, type: 1
time: 09:24, low: 166.24, high: 166.36, type: -1


1, 166.5000, 166.2000, -0.2001
09:17 09:24

time: 09:17, low: 166.36, high: 166.48, type: 1
time: 09:24, low: 166.24, high: 166.36, type: -1
time: 09:32, low: 166.12, high: 166.24, type: -1


time: 09:24, low: 166.24, high: 166.36, type: -1
time: 09:32, low: 166.12, high: 166

INFO:root:Boughtflag: -1
DEBUG:root:emotion : -3
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -1
DEBUG:root:renkoExit : True
INFO:root:exiting trade, enter: 166.95 exit: 166.85
DEBUG:root:Inside exitTrade fn
DEBUG:root:Inside addTrade to day fn
DEBUG:root:money 0.996360633753
DEBUG:root:after reset bought flag : 0
INFO:root:Short exit
INFO:root:Boughtflag: 0
INFO:root:Boughtflag: 0
INFO:root:Boughtflag: 0
INFO:root:Boughtflag: 0
INFO:root:Short trade 166.4
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -3
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -1
DEBUG:root:renkoExit : True
INFO:root:exiting trade, enter: 166.4 exit: 166.8
DEBUG:root:Inside exitTrade fn
DEBUG:root:Inside addTrade to day fn
DEBUG:root:money 0.993772485459
DEBUG:root:after reset bought flag : 0
INFO:root:Short exit
INFO:root:Boughtflag: 0
INFO:root:Boughtflag: 0
INFO:root:Boughtflag: 0
INFO:root:Boughtflag: 0
INFO:root:Boughtflag: 0
INFO:root:Boughtflag: 0
INFO:root:Long trade 166.9
INFO:root:Boughtflag: 1
DEBUG:r

time: 10:15, low: 166.84, high: 166.96, type: -1
time: 10:15, low: 166.72, high: 166.84, type: -1
time: 10:15, low: 166.6, high: 166.72, type: -1


time: 10:15, low: 166.72, high: 166.84, type: -1
time: 10:15, low: 166.6, high: 166.72, type: -1
time: 10:17, low: 166.72, high: 166.84, type: 1


-1, 166.9500, 166.8500, 0.0399
10:10 10:17

time: 10:15, low: 166.6, high: 166.72, type: -1
time: 10:17, low: 166.72, high: 166.84, type: 1
time: 10:19, low: 166.84, high: 166.96, type: 1


time: 10:17, low: 166.72, high: 166.84, type: 1
time: 10:19, low: 166.84, high: 166.96, type: 1
time: 10:24, low: 166.72, high: 166.84, type: -1


time: 10:19, low: 166.84, high: 166.96, type: 1
time: 10:24, low: 166.72, high: 166.84, type: -1
time: 10:25, low: 166.6, high: 166.72, type: -1


time: 10:24, low: 166.72, high: 166.84, type: -1
time: 10:25, low: 166.6, high: 166.72, type: -1
time: 10:26, low: 166.48, high: 166.6, type: -1


time: 10:25, low: 166.6, high: 166.72, type: -1
time: 10:26, low: 166.48, 

INFO:root:Boughtflag: 0
INFO:root:Short trade 168.4
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -3
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -1
DEBUG:root:renkoExit : True
INFO:root:exiting trade, enter: 168.4 exit: 169.05
DEBUG:root:Inside exitTrade fn
DEBUG:root:Inside addTrade to day fn
DEBUG:root:money 0.995270588796
DEBUG:root:after reset bought flag : 0
INFO:root:Short exit
INFO:root:Boughtflag: 0
INFO:root:Boughtflag: 0
INFO:root:Long trade 169.05
INFO:root:Boughtflag: 1
DEBUG:root:emotion : 3
INFO:root:Boughtflag: 1
DEBUG:root:emotion : 3
INFO:root:Boughtflag: 1
DEBUG:root:emotion : 3
INFO:root:Boughtflag: 1
DEBUG:root:emotion : 3
INFO:root:Boughtflag: 1
DEBUG:root:emotion : 3
INFO:root:Boughtflag: 1
DEBUG:root:emotion : 1
DEBUG:root:renkoExit : True
INFO:root:exiting trade, enter: 169.05 exit: 169.2
DEBUG:root:Inside exitTrade fn
DEBUG:root:Inside addTrade to day fn
DEBUG:root:money 0.99595447304
DEBUG:root:after reset bought flag : 0
INFO:root:Long exit
INFO:root:Bought

...

DEBUG:root:emotion : 3
INFO:root:Boughtflag: 1
DEBUG:root:emotion : 3
INFO:root:Boughtflag: 1
DEBUG:root:emotion : 3
INFO:root:Boughtflag: 1
DEBUG:root:emotion : 3
INFO:root:Boughtflag: 1
DEBUG:root:emotion : 3
INFO:root:Boughtflag: 1
DEBUG:root:emotion : 3
INFO:root:Boughtflag: 1
DEBUG:root:emotion : 1
DEBUG:root:renkoExit : True
INFO:root:exiting trade, enter: 195.25 exit: 193.0
DEBUG:root:Inside exitTrade fn
DEBUG:root:Inside addTrade to day fn
DEBUG:root:money 0.946091500866
DEBUG:root:after reset bought flag : 0
INFO:root:Long exit
INFO:root:Boughtflag: 0
INFO:root:Boughtflag: 0
INFO:root:Short trade 193.0
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -3
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -3
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -3
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -3
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -3
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -3
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -3
INFO:root:Boughtflag: -1
DEBUG:root:em

time: 15:32, low: 194.32, high: 194.44, type: 1
time: 15:32, low: 194.44, high: 194.56, type: 1
time: 15:32, low: 194.56, high: 194.68, type: 1


time: 15:32, low: 194.44, high: 194.56, type: 1
time: 15:32, low: 194.56, high: 194.68, type: 1
time: 15:32, low: 194.68, high: 194.8, type: 1


time: 15:32, low: 194.56, high: 194.68, type: 1
time: 15:32, low: 194.68, high: 194.8, type: 1
time: 15:32, low: 194.8, high: 194.92, type: 1


time: 15:32, low: 194.68, high: 194.8, type: 1
time: 15:32, low: 194.8, high: 194.92, type: 1
time: 15:32, low: 194.92, high: 195.04, type: 1


time: 15:32, low: 194.8, high: 194.92, type: 1
time: 15:32, low: 194.92, high: 195.04, type: 1
time: 15:32, low: 195.04, high: 195.16, type: 1


time: 15:32, low: 194.92, high: 195.04, type: 1
time: 15:32, low: 195.04, high: 195.16, type: 1
time: 15:42, low: 194.92, high: 195.04, type: -1


1, 195.2500, 193.0000, -1.1721
15:32 15:42

time: 15:32, low: 195.04, high: 195.16, type: 1
time: 15:42, low: 194.92, high: 195.0

INFO:root:Boughtflag: -1
DEBUG:root:emotion : -3
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -3
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -3
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -3
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -3
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -1
DEBUG:root:renkoExit : True
INFO:root:exiting trade, enter: 193.0 exit: 195.25
DEBUG:root:Inside exitTrade fn
DEBUG:root:Inside addTrade to day fn
DEBUG:root:money 0.913211428062
DEBUG:root:after reset bought flag : 0
INFO:root:Short exit
INFO:root:Boughtflag: 0
INFO:root:Boughtflag: 0
INFO:root:Long trade 195.25
INFO:root:Boughtflag: 1
DEBUG:root:emotion : 3
INFO:root:Boughtflag: 1
DEBUG:root:emotion : 3
INFO:root:Boughtflag: 1
DEBUG:root:emotion : 3
INFO:root:Boughtflag: 1
DEBUG:root:emotion : 3
INFO:root:Boughtflag: 1
DEBUG:root:emotion : 3
INFO:root:Boughtflag: 1
DEBUG:root:emotion : 3
INFO:root:Boughtflag: 1
DEBUG:root:emotion : 3
INFO:root:Boughtflag: 1
DEBUG:root:emotion : 3
INFO:root:Bough

time: 15:44, low: 193.72, high: 193.84, type: -1
time: 15:44, low: 193.6, high: 193.72, type: -1
time: 15:44, low: 193.48, high: 193.6, type: -1


time: 15:44, low: 193.6, high: 193.72, type: -1
time: 15:44, low: 193.48, high: 193.6, type: -1
time: 15:44, low: 193.36, high: 193.48, type: -1


time: 15:44, low: 193.48, high: 193.6, type: -1
time: 15:44, low: 193.36, high: 193.48, type: -1
time: 15:44, low: 193.24, high: 193.36, type: -1


time: 15:44, low: 193.36, high: 193.48, type: -1
time: 15:44, low: 193.24, high: 193.36, type: -1
time: 15:44, low: 193.12, high: 193.24, type: -1


time: 15:44, low: 193.24, high: 193.36, type: -1
time: 15:44, low: 193.12, high: 193.24, type: -1
time: 15:44, low: 193.0, high: 193.12, type: -1


time: 15:44, low: 193.12, high: 193.24, type: -1
time: 15:44, low: 193.0, high: 193.12, type: -1
time: 15:46, low: 193.12, high: 193.24, type: 1


-1, 193.0000, 195.2500, -1.1721
15:44 15:46

time: 15:44, low: 193.0, high: 193.12, type: -1
time: 15:46, low: 193

INFO:root:Boughtflag: 1
DEBUG:root:emotion : 3
INFO:root:Boughtflag: 1
DEBUG:root:emotion : 3
INFO:root:Boughtflag: 1
DEBUG:root:emotion : 3
INFO:root:Boughtflag: 1
DEBUG:root:emotion : 1
DEBUG:root:renkoExit : True
INFO:root:exiting trade, enter: 195.25 exit: 193.0
DEBUG:root:Inside exitTrade fn
DEBUG:root:Inside addTrade to day fn
DEBUG:root:money 0.881474055713
DEBUG:root:after reset bought flag : 0
INFO:root:Long exit
INFO:root:Boughtflag: 0
INFO:root:Boughtflag: 0
INFO:root:Short trade 193.0
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -3
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -3
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -3
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -3
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -3
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -3
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -3
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -3
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -3
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -3
INFO:roo

In [ ]:
day.money

In [ ]:
day.boughtFlag

In [ ]:
day.dailyTrades

In [ ]:
day.printAllTrades(True)